# Azure AI Search com Integração NVIDIA NIM e LlamaIndex

Neste notebook, vamos demonstrar como aproveitar os modelos de IA da NVIDIA e o LlamaIndex para criar um pipeline poderoso de Geração com Recuperação Aprimorada (RAG). Utilizaremos os LLMs e embeddings da NVIDIA, integrando-os com o Azure AI Search como o repositório vetorial, e realizaremos RAG para melhorar a qualidade e eficiência da pesquisa.

## Benefícios
- **Escalabilidade**: Aproveite os modelos de linguagem de grande escala da NVIDIA e o Azure AI Search para recuperação escalável e eficiente.
- **Eficiência de Custos**: Otimize a pesquisa e recuperação com armazenamento vetorial eficiente e técnicas de pesquisa híbrida.
- **Alto Desempenho**: Combine LLMs poderosos com pesquisa vetorizada para respostas mais rápidas e precisas.
- **Qualidade**: Mantenha alta qualidade de pesquisa fundamentando as respostas dos LLMs com documentos relevantes recuperados.

## Pré-requisitos
- 🐍 Python 3.9 ou superior
- 🔗 [Serviço Azure AI Search](https://learn.microsoft.com/azure/search/)
- 🔗 Chave de API da NVIDIA para acesso aos LLMs e Embeddings da NVIDIA via os microserviços NVIDIA NIM

## Funcionalidades Abrangidas
- ✅ Integração com LLM da NVIDIA (usaremos [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Embeddings da NVIDIA (usaremos [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Modos Avançados de Recuperação do Azure AI Search
- ✅ Indexação de Documentos com LlamaIndex
- ✅ RAG utilizando Azure AI Search e LlamaIndex com LLMs da NVIDIA

Vamos começar!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Instalação e Requisitos
Crie um ambiente Python utilizando a versão do Python >3.10.

## Começar!


Para começar, precisa de uma `NVIDIA_API_KEY` para utilizar os modelos da NVIDIA AI Foundation:  
1) Crie uma conta gratuita com [NVIDIA](https://build.nvidia.com/explore/discover).  
2) Clique no modelo da sua escolha.  
3) Em Input, selecione o separador Python e clique em **Get API Key** e, em seguida, clique em **Generate Key**.  
4) Copie e guarde a chave gerada como NVIDIA_API_KEY. A partir daí, terá acesso aos endpoints.  


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Exemplo de RAG usando LLM e Embedding
### 1) Inicializar o LLM
`llama-index-llms-nvidia`, também conhecido como conector LLM da NVIDIA, permite que você se conecte e gere a partir de modelos compatíveis disponíveis no catálogo de APIs da NVIDIA. Veja aqui uma lista de modelos de conclusão de chat: https://build.nvidia.com/search?term=Text-to-Text

Aqui iremos utilizar **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Inicializar a Embedding
`llama-index-embeddings-nvidia`, também conhecido como conector de Embeddings da NVIDIA, permite que você se conecte e gere a partir de modelos compatíveis disponíveis no catálogo de APIs da NVIDIA. Selecionámos `nvidia/nv-embedqa-e5-v5` como o modelo de embedding. Veja aqui uma lista de modelos de embedding de texto: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Criar um Armazenamento Vetorial de Pesquisa Azure AI


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Criar um Motor de Consulta para fazer perguntas sobre os seus dados

Aqui está uma consulta utilizando pesquisa vetorial pura no Azure AI Search e fundamentando a resposta no nosso LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Aqui está uma consulta usando pesquisa híbrida no Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Análise de Pesquisa Vetorial
A resposta do LLM captura com precisão as principais consequências económicas mencionadas no texto fonte sobre o mercado de ações da Rússia. Especificamente, afirma que o mercado de ações russo sofreu uma queda significativa, perdendo 40% do seu valor, e que as negociações foram suspensas devido à situação em curso. Esta resposta está bem alinhada com as informações fornecidas na fonte, indicando que o LLM identificou e resumiu corretamente os detalhes relevantes sobre o impacto no mercado de ações como resultado das ações da Rússia e das sanções impostas.

#### Comentário sobre os Nós Fonte
Os nós fonte fornecem um relato detalhado das consequências económicas que a Rússia enfrentou devido às sanções internacionais. O texto destaca que o mercado de ações russo perdeu 40% do seu valor e que as negociações foram suspensas. Além disso, menciona outras repercussões económicas, como a desvalorização do Rublo e o isolamento mais amplo da economia russa. A resposta do LLM conseguiu sintetizar eficazmente os pontos críticos destes nós, concentrando-se no impacto no mercado de ações conforme solicitado pela consulta.


Agora, vamos analisar uma consulta em que a Pesquisa Híbrida não fornece uma resposta bem fundamentada:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Pesquisa Híbrida: Análise da Resposta do LLM
A resposta do LLM no exemplo de Pesquisa Híbrida indica que o contexto fornecido não especifica a data exata da invasão da Ucrânia pela Rússia. Esta resposta sugere que o LLM está a utilizar as informações disponíveis nos documentos de origem, mas reconhece a ausência de detalhes precisos no texto.

A resposta é precisa ao identificar que o contexto menciona eventos relacionados à agressão da Rússia, mas não determina a data específica da invasão. Isto demonstra a capacidade do LLM de compreender as informações fornecidas enquanto reconhece lacunas no conteúdo. O LLM incentiva eficazmente o utilizador a procurar fontes externas ou registos históricos para obter a data exata, exibindo um nível de cautela quando as informações estão incompletas.

### Análise dos Nós de Origem
Os nós de origem no exemplo de Pesquisa Híbrida contêm excertos de um discurso que aborda a resposta dos EUA às ações da Rússia na Ucrânia. Estes nós destacam o impacto geopolítico mais amplo e as medidas tomadas pelos EUA e seus aliados em resposta à invasão, mas não mencionam a data específica da invasão. Isto está alinhado com a resposta do LLM, que identifica corretamente que o contexto não contém informações precisas sobre a data.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Análise Híbrida com Reclassificação: Resposta do LLM
No exemplo de análise híbrida com reclassificação, a resposta do LLM fornece contexto adicional ao mencionar que o evento ocorreu seis dias antes do discurso ser feito. Isto indica que o LLM é capaz de inferir a data da invasão com base no momento do discurso, embora ainda seja necessário conhecer a data exata do discurso para maior precisão.

Esta resposta demonstra uma capacidade aprimorada de usar pistas contextuais para fornecer uma resposta mais informativa. Destaca a vantagem da reclassificação, onde o LLM pode acessar e priorizar informações mais relevantes para oferecer uma aproximação mais precisa do detalhe desejado (ou seja, a data da invasão).

### Análise dos Nós de Origem
Os nós de origem neste exemplo incluem referências ao momento da invasão da Rússia, mencionando especificamente que ocorreu seis dias antes do discurso. Embora a data exata ainda não seja explicitamente indicada, os nós fornecem um contexto temporal que permite ao LLM oferecer uma resposta mais detalhada. A inclusão deste detalhe demonstra como a reclassificação pode melhorar a capacidade do LLM de extrair e inferir informações a partir do contexto fornecido, resultando numa resposta mais precisa e informativa.


**Nota:**  
Neste notebook, utilizámos microserviços NVIDIA NIM do Catálogo de APIs da NVIDIA.  
As APIs mencionadas acima, `NVIDIA (llms)`, `NVIDIAEmbedding`, e [Azure AI Search Semantic Hybrid Retrieval (com reranking incorporado)](https://learn.microsoft.com/azure/search/semantic-search-overview). Note que as APIs mencionadas acima também podem suportar microserviços auto-hospedados.  

**Exemplo:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
